In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medquad-medical-question-answer-for-ai-research/medquad.csv
/kaggle/input/stanford-question-answering-dataset/train-v1.1.json
/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model.safetensors.index.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00003-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/merges.txt
/kaggle/input/qwen2/transformers/7b-instruct/1/LICENSE
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00001-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/README.md
/kaggle/input/qwen2/transformers/7b-instruct/1/tokenizer.json
/kaggle/input/qwen2/transformers/7b-instruct/1/vocab.json
/kaggle/input/qwen2/transformers/7b-instruct/1/tokenizer_config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00004-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00002-of-

In [ ]:
!pip install --quiet transformers sentence-transformers pinecone tqdm accelerate


In [ ]:
!pip install protobuf==3.20.3 --quiet --force-reinstall


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
pinecone_api = user_secrets.get_secret("pinecone_api_key")

In [1]:
import pandas as pd
import json
import pinecone
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import numpy as np
from tqdm.auto import tqdm
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'pinecone'

In [68]:
# Initialize Pinecone client
pc = Pinecone(api_key=pinecone_api)

index_name = "rag-project"
dimension = 768  # for all-mpnet-base-v2 embeddings
metric = "cosine"

# Check if index exists
existing_indexes = [i["name"] for i in pc.list_indexes()]
print("Existing indexes:", existing_indexes)

if index_name not in existing_indexes:
    print("Creating index...")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(index_name)
print(f"Connected to Pinecone index: {index_name}")

Existing indexes: ['rag-project']
Connected to Pinecone index: rag-project


In [14]:
# --- MedQuad (Medical QA) ---
medquad = pd.read_csv("/kaggle/input/medquad-medical-question-answer-for-ai-research/medquad.csv")
medquad_docs = medquad['question'].tolist()
medquad_answers = medquad['answer'].tolist()
medquad_domain = ['Medical'] * len(medquad_docs)

# --- Amazon QA (E-commerce) ---
amazon_multi_q = pd.read_csv("/kaggle/input/amazon-questionanswer-dataset/multi_questions.csv")
amazon_single = pd.read_csv("/kaggle/input/amazon-questionanswer-dataset/single_qna.csv")
amazon_multi_a = pd.read_csv("/kaggle/input/amazon-questionanswer-dataset/multi_answers.csv")

amazon_docs = (
    amazon_multi_q['QuestionText'].tolist() +
    amazon_single['Question'].tolist() +
    amazon_multi_a['AnswerText'].tolist()
)
amazon_answers = amazon_single['Answer'].tolist() + amazon_multi_a['AnswerText'].tolist()
amazon_domain = ['E-commerce'] * len(amazon_docs)

# --- SQuAD (General QA) ---
def load_squad(path):
    with open(path) as f:
        data = json.load(f)
    docs, answers = [], []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                docs.append(qa['question'])
                answers.append(qa['answers'][0]['text'] if qa['answers'] else "")
    return docs, answers

squad_docs, squad_answers = load_squad("/kaggle/input/stanford-question-answering-dataset/train-v1.1.json")
squad_domain = ['General'] * len(squad_docs)

# --- CISI (Information Retrieval) ---
with open("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL", "r") as f:
    cisi_docs = [line.strip() for line in f.readlines() if line.strip()]
cisi_answers = [""] * len(cisi_docs)
cisi_domain = ['IR'] * len(cisi_docs)

# --- Combine all ---
all_docs = medquad_docs + amazon_docs + squad_docs + cisi_docs
all_answers = medquad_answers + amazon_answers + squad_answers + cisi_answers
all_domains = medquad_domain + amazon_domain + squad_domain + cisi_domain

print(f"✅ Total documents: {len(all_docs)}")
print(f"✅ Domains: {set(all_domains)}")

Total documents: 5801972
Total answers: 5629355
Domains: {'Medical', 'E-commerce', 'IR', 'General'}


In [ ]:
# Sample only 10% (adjustable)
sample_fraction = 0.1
sample_indices = random.sample(range(len(all_docs)), int(len(all_docs) * sample_fraction))
sampled_docs = [all_docs[i] for i in sample_indices]
sampled_answers = [all_answers[i] for i in sample_indices]
sampled_domains = [all_domains[i] for i in sample_indices]

print(f"🧩 Sampled {len(sampled_docs)} docs ({sample_fraction*100:.0f}%)")

# Embed using smaller model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sampled_docs, show_progress_bar=True, batch_size=64)

# PCA to compress from 384 → 256 dims
pca = PCA(n_components=256)
embeddings_pca = pca.fit_transform(embeddings)

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

batch_size = 100
for batch_ids, batch_embeds, batch_docs, batch_answers, batch_domains in tqdm(
    zip(
        chunks(range(len(embeddings_pca)), batch_size),
        chunks(embeddings_pca, batch_size),
        chunks(sampled_docs, batch_size),
        chunks(sampled_answers, batch_size),
        chunks(sampled_domains, batch_size)
    ),
    total=len(embeddings_pca) // batch_size
):
    vectors = [
        {
            "id": f"doc-{i}",
            "values": emb,
            "metadata": {
                "text": doc,
                "answer": ans,
                "domain": domain
            }
        }
        for i, emb, doc, ans, domain in zip(batch_ids, batch_embeds, batch_docs, batch_answers, batch_domains)
    ]
    index.upsert(vectors=vectors)

print("✅ All embeddings upserted successfully!")

 11%|█         | 9585/90656 [1:21:13<10:57:47,  2.05it/s]

In [ ]:
gen_model_path = "/kaggle/input/qwen2/transformers/7b-instruct/1"
gen_pipe = pipeline(
    "text-generation",
    model=gen_model_path,
    device=0,       # GPU=0, CPU=-1
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)


In [ ]:
def rag_answer_self_consistent(query, top_k=5, num_samples=5):
    # 1️⃣ Embed query
    query_vec = embed_model.encode(query).tolist()
    
    # 2️⃣ Retrieve from Pinecone
    results = index.query(vector=query_vec, top_k=top_k)
    context = " ".join([match.id for match in results.matches])
    
    # 3️⃣ Generate multiple answers
    prompt = f"Answer the question based on context:\nQuestion: {query}\nContext: {context}"
    answers = [gen_pipe(prompt)[0]['generated_text'] for _ in range(num_samples)]
    
    # 4️⃣ Majority vote for final answer
    most_common = Counter(answers).most_common(1)[0][0]
    return most_common, answers

In [ ]:
def compute_f1(pred, gold):
    pred_tokens = pred.split()
    gold_tokens = gold.split()
    common = set(pred_tokens) & set(gold_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    if precision + recall == 0:
        return 0.0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def exact_match(pred, gold):
    return int(pred.strip().lower() == gold.strip().lower())

In [ ]:
domain_metrics = {}

for domain in set(all_domains):
    print(f"Evaluating domain: {domain}")
    indices = [i for i, d in enumerate(all_domains) if d==domain and all_answers[i]!=""]
    em_scores, f1_scores, self_consistency = [], [], []
    
    for idx in tqdm(indices):
        query = all_docs[idx]
        gold = all_answers[idx]
        
        pred, all_preds = rag_answer_self_consistent(query, top_k=5, num_samples=5)
        em_scores.append(exact_match(pred, gold))
        f1_scores.append(compute_f1(pred, gold))
        
        # Self-consistency: fraction of identical answers
        counts = Counter(all_preds)
        sc = counts.most_common(1)[0][1] / len(all_preds)
        self_consistency.append(sc)
    
    domain_metrics[domain] = {
        "ExactMatch": sum(em_scores)/len(em_scores)*100 if em_scores else None,
        "F1": sum(f1_scores)/len(f1_scores)*100 if f1_scores else None,
        "SelfConsistency": sum(self_consistency)/len(self_consistency)*100 if self_consistency else None,
        "NumQueries": len(indices)
    }

In [ ]:
for domain, metrics in domain_metrics.items():
    print(f"\nDomain: {domain}")
    print(f"NumQueries: {metrics['NumQueries']}")
    print(f"Exact Match (%): {metrics['ExactMatch']}")
    print(f"F1 (%): {metrics['F1']}")
    print(f"Self-Consistency (%): {metrics['SelfConsistency']}")